# Warsztat 2: Lakeflow Pipelines & Databricks Jobs Orchestration

**Cel warsztatu:**
- Poznanie Delta Live Tables (DLT) - deklaratywnych pipeline'ów
- Implementacja Data Quality z Expectations
- Konfiguracja Multi-task Databricks Jobs
- Parametryzacja i dependency management

**Czas:** 90 minut

**Uwaga:** Ten warsztat wymaga UI Databricks dla Delta Live Tables i Jobs.

---

## 📚 Inicjalizacja środowiska

In [ ]:
%run ../../00_setup

## 🎯 Część 1: Delta Live Tables - Podstawy

### Co to jest Delta Live Tables?

**Delta Live Tables (DLT)** to deklaratywny framework do budowania reliable data pipelines.

**Kluczowe koncepcje:**
- **Deklaratywne** - opisujesz CO, nie JAK
- **Streaming & Batch** - jedna składnia dla obu
- **Data Quality** - wbudowane expectations
- **Auto-scaling** - automatyczne zarządzanie zasobami
- **Lineage** - automatyczne śledzenie zależności

### Zadanie 1.1: Przygotowanie DLT Pipeline Definition

**Instrukcje:**
1. Utwórz nowy notebook dla DLT pipeline
2. Zdefiniuj Bronze, Silver, Gold tables używając dekoratora `@dlt.table`
3. Dodaj expectations dla data quality

**Przykładowy kod DLT:**

In [ ]:
# TODO: Zadanie 1.1 - DLT Pipeline Definition
# Ten kod musi być uruchomiony w DLT pipeline (nie w zwykłym notebooku)

import dlt
from pyspark.sql.functions import *

# Bronze Table - raw data
@dlt.table(
    name="____",  # bronze_customers
    comment="Raw customer data from source",
    table_properties={
        "quality": "bronze"
    }
)
def bronze_customers():
    # Użyj ścieżki Volume jako źródła danych
    volume_path = "/Volumes/main/default/kion_data"
    
    return (
        spark.readStream
        .format("____")  # cloudFiles
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .load(f"____")  # {volume_path}/customers/
        .withColumn("_ingest_timestamp", current_timestamp())
    )

# Silver Table - cleaned data z expectations
@dlt.table(
    name="silver_customers",
    comment="Cleaned customer data with quality checks"
)
@dlt.expect_or_drop("____", "____ IS NOT NULL")  # valid_email, email
@dlt.expect_or_drop("valid_customer_id", "customer_id > 0")
def silver_customers():
    return (
        dlt.read_stream("____")  # bronze_customers
        .select(
            "customer_id",
            "name",
            "email",
            "city",
            upper(col("country")).alias("country")
        )
    )

# Gold Table - aggregated data
@dlt.table(
    name="____",  # gold_customers_by_country
    comment="Customer count per country"
)
def gold_customers_by_country():
    return (
        dlt.read("____")  # silver_customers
        .groupBy("____")  # country
        .agg(count("____").alias("total_customers"))  # customer_id
    )

### Zadanie 1.2: DLT Expectations - Data Quality

**Typy Expectations:**

1. **`@dlt.expect()`** - Loguj naruszenia, ale kontynuuj
2. **`@dlt.expect_or_drop()`** - Odrzuć rekordy nie spełniające warunku
3. **`@dlt.expect_or_fail()`** - Zatrzymaj pipeline przy naruszeniu

**Instrukcje:**
1. Dodaj expectations do Silver table:
   - Email musi zawierać '@'
   - Customer_id > 0
   - Country nie może być NULL
2. Użyj różnych typów expectations (drop, fail, log)

**Przykład:**

In [ ]:
# TODO: Zadanie 1.2 - Expectations examples
# (Ten kod jest przykładem - musi być w DLT pipeline)

@dlt.table(name="silver_orders")
@dlt.____("____", "____ IS NOT NULL")  # expect, valid_order_id, order_id
@dlt.expect_or_drop("____", "____ > 0")  # positive_amount, total_amount
@dlt.expect_or_fail("____", "status IN ('____', '____', '____')")  # valid_status, pending, completed, cancelled
def silver_orders():
    # Źródło danych z Volume
    volume_path = "/Volumes/main/default/kion_data"
    
    return (
        dlt.read_stream("____")  # bronze_orders - lub użyj cloudFiles z volume_path/orders/
        .select(
            "order_id",
            "customer_id",
            to_date(col("order_date")).alias("order_date"),
            col("total_amount").cast("double"),
            "status"
        )
    )

### Zadanie 1.3: DLT Configuration & Deployment

**Instrukcje (UI Databricks):**

1. Przejdź do **Workflows** → **Delta Live Tables**
2. Kliknij **Create Pipeline**
3. Konfiguracja:
   - **Pipeline name**: `workshop_dlt_pipeline`
   - **Notebook libraries**: Dodaj notebook z definicją DLT
   - **Target**: `{CATALOG}.{SCHEMA}`
   - **Storage location**: `/mnt/dlt/{user}/`
   - **Pipeline mode**: `Triggered` lub `Continuous`
   - **Cluster mode**: `Fixed size` (1 worker dla testów)
4. Kliknij **Create**
5. Kliknij **Start** aby uruchomić pipeline

**Oczekiwany rezultat:**
- Pipeline utworzony i uruchomiony
- Tabele Bronze/Silver/Gold utworzone
- Data quality metrics widoczne w UI

**Zadanie do wykonania:**
- Stwórz DLT pipeline w UI
- Zrzut ekranu z uruchomionego pipeline
- Sprawdź Event Log dla metryki quality

---

## 🔗 Część 2: Databricks Jobs - Multi-task Orchestration

### Co to są Databricks Jobs?

**Databricks Jobs** - orkiestracja multi-task workflows.

**Typy tasków:**
- **Notebook task** - wykonaj notebook
- **DLT task** - uruchom DLT pipeline
- **SQL task** - wykonaj SQL query
- **Python wheel task** - uruchom Python package
- **JAR task** - uruchom Java/Scala

### Zadanie 2.1: Widget Parameters - Parametryzacja notebooków

**Instrukcje:**
1. Użyj `dbutils.widgets` do zdefiniowania parametrów
2. Odbierz parametry z Job execution
3. Użyj parametrów w transformacjach

In [ ]:
# TODO: Zadanie 2.1 - Widget parameters

# Definicja widgets
dbutils.widgets.text("____", "____", "Start Date (YYYY-MM-DD)")  # start_date, 2024-01-01
dbutils.widgets.text("____", "2024-12-31", "End Date (YYYY-MM-DD)")  # end_date
dbutils.widgets.dropdown("environment", "dev", ["dev", "____", "____"], "Environment")  # staging, prod

# Odczyt parametrów
start_date = dbutils.widgets.get("____")  # start_date
end_date = dbutils.widgets.get("end_date")
environment = dbutils.widgets.get("____")  # environment

print(f"=== Job Parameters ===")
print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")
print(f"Environment: {environment}")

# Użyj parametrów w query - wczytaj dane z Volume
volume_path = "/Volumes/main/default/kion_data"
orders_from_volume = spark.read.json(f"{volume_path}/orders/orders_batch.json")

# Filtruj używając parametrów
filtered_orders = orders_from_volume.filter(
    (F.col("order_date") >= start_date) & 
    (F.col("order_date") <= end_date)
)

# Alternatywnie, jeśli dane już są w tabelach:
# filtered_orders = spark.sql(f"""
#     SELECT * 
#     FROM {CATALOG}.{SCHEMA}.orders
#     WHERE order_date BETWEEN '{start_date}' AND '{____}'  -- end_date
# """)

print(f"Filtered orders: {filtered_orders.count()}")

### Zadanie 2.2: Multi-task Job Configuration

**Instrukcje (UI Databricks):**

1. Przejdź do **Workflows** → **Jobs**
2. Kliknij **Create Job**
3. Dodaj pierwszy task:
   - **Task name**: `ingest_bronze`
   - **Type**: `Notebook`
   - **Source**: Wybierz notebook dla Bronze ingestion
   - **Cluster**: `New job cluster` (1 worker)
   - **Parameters**: `{"start_date": "2024-01-01"}`
4. Dodaj drugi task (depends on: `ingest_bronze`):
   - **Task name**: `transform_silver`
   - **Type**: `Notebook`
   - **Source**: Notebook dla Silver transformations
   - **Depends on**: `ingest_bronze`
5. Dodaj trzeci task (depends on: `transform_silver`):
   - **Task name**: `aggregate_gold`
   - **Type**: `Notebook`
   - **Depends on**: `transform_silver`
6. Skonfiguruj **Schedule**: `Cron: 0 2 * * *` (2 AM daily)
7. Dodaj **Email notifications** na failure
8. Kliknij **Create**

**DAG Structure:**
```
ingest_bronze
      ↓
transform_silver
      ↓
aggregate_gold
```

**Zadanie do wykonania:**
- Utwórz Multi-task Job w UI
- Zdefiniuj dependencies między taskami
- Uruchom Job ręcznie (Run Now)
- Sprawdź Runs history

### Zadanie 2.3: Job Dependencies & Conditional Execution

**Instrukcje:**
1. Dodaj task z warunkiem (If/Else logic):
   - Jeśli Bronze ma > 1000 rekordów → uruchom Silver
   - Jeśli nie → wyślij alert
2. Użyj task values do przekazania danych między taskami

In [ ]:
# TODO: Zadanie 2.3 - Task values (przekazywanie danych)

# Task 1: Sprawdź liczbę rekordów
bronze_count = spark.table(f"{CATALOG}.{SCHEMA}.bronze_customers").count()

print(f"Bronze records: {bronze_count}")

# Ustaw task value (dostępne dla następnych tasków)
dbutils.jobs.taskValues.set(
    key="____",  # bronze_count
    value=bronze_count
)

# Task 2: Odbierz wartość z poprzedniego taska
# received_count = dbutils.jobs.taskValues.get(
#     taskKey="ingest_bronze",
#     key="bronze_count",
#     default=0
# )

# if received_count > 1000:
#     print("✓ Sufficient data - proceeding to Silver")
# else:
#     raise Exception(f"Insufficient data: {received_count} records")

### Zadanie 2.4: Job Monitoring & Alerting

**Instrukcje (UI Databricks):**

1. W Job configuration, przejdź do **Alerts**
2. Dodaj email notification:
   - **On failure**: `your-email@example.com`
   - **On success**: (opcjonalnie)
3. Skonfiguruj **Retries**:
   - **Max retries**: 3
   - **Timeout**: 3600 seconds (1 hour)
4. Dodaj **SLA**:
   - Jeśli Job trwa > 2 godziny → alert

**Monitoring w kodzie:**

In [ ]:
# TODO: Zadanie 2.4 - Job monitoring code

import time

# Start timer
start_time = time.time()

try:
    # Twój kod transformacji
    print("Processing data...")
    time.sleep(2)  # Symulacja przetwarzania
    
    # Zapisz metrics
    end_time = time.time()
    duration = end_time - start_time
    
    dbutils.jobs.taskValues.set("____", duration)  # execution_time
    
    print(f"✓ Success! Duration: {duration:.2f}s")
    
except Exception as e:
    # Log error
    print(f"✗ Error: {str(e)}")
    
    # Ustaw failure reason
    dbutils.jobs.taskValues.set("failure_reason", str(e))
    
    # Re-raise dla Job failure
    raise

---

## ✅ Podsumowanie warsztatu

**Zrealizowane cele:**
- ✅ Delta Live Tables - deklaratywne pipeline definition
- ✅ Data Quality z Expectations (drop, fail, log)
- ✅ Multi-task Databricks Jobs z dependencies
- ✅ Parametryzacja i task values
- ✅ Monitoring i alerting

**Kluczowe wnioski:**

1. **DLT vs Traditional Spark:**
   - DLT: Deklaratywne, auto-scaling, built-in quality
   - Spark: Imperatywne, manual scaling, custom quality

2. **Kiedy używać DLT:**
   - Production data pipelines
   - Data quality critical
   - Need auto-scaling
   - Want automatic lineage

3. **Databricks Jobs Best Practices:**
   - Małe, reusable notebooki (single responsibility)
   - Parametryzacja przez widgets
   - Task dependencies dla DAG
   - Monitoring i alerting dla critical jobs

**Quick Reference:**

| Feature | DLT | Traditional Spark |
|---------|-----|------------------|
| Składnia | Deklaratywna | Imperatywna |
| Data Quality | Built-in expectations | Custom code |
| Lineage | Automatic | Manual |
| Orchestration | Automatic | Manual (Jobs) |
| Scaling | Auto | Manual |

**Następne kroki:**
- **Kolejny warsztat**: 03_governance_integrations_workshop.ipynb
- **Dokumentacja**: [DLT Guide](https://docs.databricks.com/delta-live-tables/)
- **Best Practices**: [Databricks Jobs](https://docs.databricks.com/workflows/jobs/)

---

## 🧹 Cleanup (opcjonalnie)

In [ ]:
# Usuń widgets
# dbutils.widgets.removeAll()

# Wyczyść cache
# spark.catalog.clearCache()